# Week Five Assignment 1: Exploring Lasso with scikit learn

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from math import log, sqrt
from sklearn.linear_model import Lasso
%matplotlib inline

In [6]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
train_df=pd.read_csv("wk3_kc_house_train_data.csv",dtype=dtype_dict)
test_df=pd.read_csv("wk3_kc_house_test_data.csv",dtype=dtype_dict)
valid_df=pd.read_csv("wk3_kc_house_valid_data.csv",dtype=dtype_dict)

#Create new fetures for sales df
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

In [134]:
#Create lasso object with L1 penalty of 5e2
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']
lasso1=Lasso(alpha=5e2,normalize=True).fit(sales[all_features],sales.price)

In [135]:
def rss(ytrue,ypredict):
    return ((ytrue-ypredict)**2).sum()

## Task 1
Which features have been selected by Lasso?

In [138]:
lasso1.coef_

array([     0.        ,      0.        ,      0.        ,    134.43931396,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,  24750.00458561,      0.        ,
        61749.10309071,      0.        ,      0.        ,     -0.        ,
            0.        ])

In [136]:
list(lasso1.coef_[lasso1.coef_!=0])

[134.43931395541435, 24750.004585609502, 61749.103090708129]

In [137]:
nonzero_coef=list(lasso1.coef_[lasso1.coef_!=0])
lasso_features=[all_features[list(lasso1.coef_).index(element)] for element in nonzero_coef]
lasso_features

['sqft_living', 'view', 'grade']

# Task 2
Find the optimum L1 penalty for each l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type np.logspace(1, 7, num=13). Use the train, validation, and test sets given.

In [31]:
l1_penalty=np.logspace(1,7,num=13)

In [39]:
#Add features to train, valid and test sets

#Add to train...
train_df['sqft_living_sqrt'] = train_df['sqft_living'].apply(sqrt)
train_df['sqft_lot_sqrt'] = train_df['sqft_lot'].apply(sqrt)
train_df['bedrooms_square'] = train_df['bedrooms']*train_df['bedrooms']
train_df['floors_square'] = train_df['floors']*train_df['floors']

#Add to test...
test_df['sqft_living_sqrt'] = test_df['sqft_living'].apply(sqrt)
test_df['sqft_lot_sqrt'] = test_df['sqft_lot'].apply(sqrt)
test_df['bedrooms_square'] = test_df['bedrooms']*test_df['bedrooms']
test_df['floors_square'] = test_df['floors']*test_df['floors']

#Add to validation...
valid_df['sqft_living_sqrt'] = valid_df['sqft_living'].apply(sqrt)
valid_df['sqft_lot_sqrt'] = valid_df['sqft_lot'].apply(sqrt)
valid_df['bedrooms_square'] = valid_df['bedrooms']*valid_df['bedrooms']
valid_df['floors_square'] = valid_df['floors']*valid_df['floors']

In [51]:
#Train lasso models, find rss on validation set. Let's also determine the non zero coefficients with each 
#model
penalty_rss=[]
for penalty in l1_penalty:
    model=Lasso(alpha=penalty,normalize=True).fit(train_df[all_features],train_df.price)
    error=rss(valid_df.price,model.predict(valid_df[all_features]))
    penalty_rss.append((error,penalty,len(model.coef_[model.coef_!=0]),model.intercept_))

In [52]:
penalty_rss.sort()
penalty_rss

[(398213327300134.06, 10.0, 14, 6630155.6686283601),
 (399041900253348.0, 31.622776601683793, 14, 6292803.6955100335),
 (429791604072558.4, 100.0, 10, 5231990.7247574786),
 (463739831045119.9, 316.22776601683796, 5, 2612357.881978082),
 (645898733633803.2, 1000.0, 3, -118305.10802037653),
 (1222506859427156.8, 3162.2776601683795, 0, 542734.95164429874),
 (1222506859427156.8, 10000.0, 0, 542734.95164429874),
 (1222506859427156.8, 31622.776601683792, 0, 542734.95164429874),
 (1222506859427156.8, 100000.0, 0, 542734.95164429874),
 (1222506859427156.8, 316227.76601683791, 0, 542734.95164429874),
 (1222506859427156.8, 1000000.0, 0, 542734.95164429874),
 (1222506859427156.8, 3162277.6601683795, 0, 542734.95164429874),
 (1222506859427156.8, 10000000.0, 0, 542734.95164429874)]

It looks like the lowest penalty, 10, has the least validation error. This is not surprising. These penalties were quite large, causing some features to not be used. Less features may mean less accuracy. Also, we remember that increasing the penalty can reduce overfitting, thereby increasing accuracy on the validation set, but it also trains the model to minimize the lasso cost function and not the rss.

By the time the L1 penalty reaches 1000, all coefficients are zero, except for the intercept, which is non zero in all cases.

Let us now choose the model with l1 penalty as the optimum model and check the test rss

In [53]:
lasso10=Lasso(alpha=10,normalize=True).fit(train_df[all_features],train_df.price)


In [54]:
rss(test_df.price,lasso10.predict(test_df[all_features]))

98467402552698.88

## Task 3

Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero.


From above, we see that the L1 penalty of 10 has the best validation score. There are 14 nonzero coefficients and a non zero intercept. Thus, we have a total of 15 non zero coefficients.

In [59]:
#More systematically, we can use the following code to count non zero coefficients
def coef_counter(model):
    return np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

In [60]:
coef_counter(lasso10)

15

## Task 4

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

You are going to implement a simple, two phase procedure to achieve this goal:

Explore a large range of ‘l1_penalty’ values to find a narrow region of ‘l1_penalty’ values where models are likely to have the desired number of non-zero weights.
    
Further explore the narrow region you found to find a good value for ‘l1_penalty’ that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for ‘l1_penalty’.

In [83]:
alpha_num_nonzero=[]
alpha_search1=np.logspace(1,4,num=20)
for penalty in alpha_search1:
    model=Lasso(alpha=penalty,normalize=True).fit(train_df[all_features],train_df.price)
    alpha_num_nonzero.append((penalty,coef_counter(model)))

In [84]:
alpha_num_nonzero

[(10.0, 15),
 (14.384498882876629, 15),
 (20.691380811147901, 15),
 (29.763514416313178, 15),
 (42.813323987193932, 13),
 (61.584821106602639, 12),
 (88.586679041008225, 11),
 (127.42749857031335, 10),
 (183.29807108324357, 7),
 (263.66508987303581, 6),
 (379.26901907322497, 6),
 (545.55947811685144, 6),
 (784.75997035146065, 5),
 (1128.8378916846884, 3),
 (1623.776739188721, 3),
 (2335.7214690901214, 2),
 (3359.8182862837812, 1),
 (4832.9302385717519, 1),
 (6951.9279617756056, 1),
 (10000.0, 1)]

In [107]:
desired_coef=7

In [108]:
#The minimum L1 penalty is the largest alpha that has more nonzeros than desired_alpha.
#The maximum L1 penalty is the smallest alpha that has fewer nonzeros than desired_alpha
test_list=[tup for tup in alpha_num_nonzero if tup[1]>desired_coef]
test_list.sort()
l1_penalty_min=test_list[-1][0]   

test_list2=[tup for tup in alpha_num_nonzero if tup[1]<desired_coef]
test_list2.sort()
l1_penalty_max=test_list2[0][0]

In [109]:
l1_penalty_min

127.42749857031335

In [110]:
l1_penalty_max

263.66508987303581

## Task 5
Search over a range of alpha values between the min and max values found previously. Fit models on the training
data and find the rss of the prediction of the validation set. Of the models that have the desired number of coefficients, chose the model with the lowest rss

In [117]:
optimizing_list=[]
for penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model=Lasso(alpha=penalty,normalize=True).fit(train_df[all_features],train_df.price)
    RSS=rss(valid_df.price,model.predict(valid_df[all_features]))
    optimizing_list.append((RSS,penalty,coef_counter(model)))

In [121]:
opt_list2=[tup for tup in optimizing_list if tup[2]==7]
opt_list2.sort()

In [123]:
optimum_rss=opt_list2[0][0]
optimum_alpha=opt_list2[0][1]

In [124]:
print("Optimum alpha: {}\nOptimum rss: {}".format(optimum_alpha,optimum_rss))

Optimum alpha: 156.10909673930755
Optimum rss: 440037365263317.0


In [125]:
#Redefine the model with this alpha
optimum_lasso=Lasso(alpha=optimum_alpha,normalize=True).fit(train_df[all_features],train_df.price)

In [126]:
optimum_lasso.coef_

array([ -0.00000000e+00,  -0.00000000e+00,   1.06108903e+04,
         1.63380252e+02,   0.00000000e+00,  -0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         5.06451687e+05,   4.19600436e+04,   0.00000000e+00,
         1.16253554e+05,   0.00000000e+00,   0.00000000e+00,
        -2.61223488e+03,   0.00000000e+00])

In [127]:
optimum_lasso.intercept_

4422190.2791203531

## Task 6 
What features were selected in our optimum lasso model with 7 features?

In [133]:
for item in optimum_lasso.coef_[optimum_lasso.coef_!=0]:
    print(all_features[list(optimum_lasso.coef_).index(item)])

bathrooms
sqft_living
waterfront
view
grade
yr_built
